 Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests

In [2]:
import re
import pandas as pd

In [3]:
search_s = input('Введите название вакансии')
search = re.sub(r'\s', '+', search_s)
main_link = f'https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text={search}&salary='
print(main_link)

Введите название вакансииврач стоматолог
https://hh.ru/search/vacancy?area=1&clusters=true&enable_snippets=true&order_by=publication_time&text=врач+стоматолог&salary=


In [4]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
response = requests.get(main_link, headers = header).text
html = bs(response,'lxml')

In [5]:
def page_table(link):
    result = pd.DataFrame()

    global_name = 'https://hh.ru'
    main_link_last = f'{global_name}{link}'
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')

    names = html_last.findAll('div',{'class':'resume-search-item__name'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys = html_last.findAll('div',{'class':'vacancy-serp-item__row vacancy-serp-item__row_header'})
    salarys_all = []
    for i in salarys:
        salarys_2 = i.find('div',{'class':'vacancy-serp-item__compensation'})
        salarys_all.append(salarys_2)
    for i in range(len(salarys_all)):
        if salarys_all[i]:
            salarys_all[i] = salarys_all[i].getText()
    salarys_1 = []
    salarys_2 = []
    for i in range(len(salarys_all)):
        if salarys_all[i]:
            hm = re.findall(r'^от_*', salarys_all[i])
            if hm:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'от ', '', hm_num)
                salarys_1.append(hm_num)
                salarys_2.append('None')
            hm2 = re.findall(r'^до_*', salarys_all[i])
            if hm2:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'до ', '', hm_num)
                salarys_1.append('None')
                salarys_2.append(hm_num)
            hm3 = re.search(r'-', salarys_all[i])
            if hm3:
                hm_num = re.split(r'-', salarys_all[i])
                hm_num1 = re.sub(r'\xa0', '', hm_num[0])
                hm_num2 = re.sub(r'\xa0', '', hm_num[1])
                salarys_1.append(hm_num1)
                salarys_2.append(hm_num2)
        else:
            salarys_1.append('None')
            salarys_2.append('None')

    links_page = []
    for i in salarys:
        links_page1 = i.find('a',{'class':'bloko-link HH-LinkModifier'})
        links_page2 = links_page1['href']
        links_page.append(links_page2)
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [6]:
pager = html.findAll('span',{'class':'pager-item-not-in-short-range'})
last_page = pager[-1].find('a',{'class':'bloko-button HH-Pager-Control', 'data-qa': 'pager-page'})
link_last_num = last_page.getText()

In [7]:
page_other = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'links', 'from'])
for i in range(int(link_last_num)):
    link_last = f'/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&order_by=publication_time&search_field=name&text={search}&page={i}'
    result = page_table(link_last)
    page_other = pd.merge(page_other, result, how='outer')

In [8]:
import transliterate

In [9]:
search_two = transliterate.translit(search_s, reversed=True)
search_two = re.sub(r'\s', '-', search_two)
main_link_two = f'https://superjob.ru/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4'

In [10]:
def page_table_two(link):
    result = pd.DataFrame()

    global_name = 'https://superjob.ru'
    main_link_last = f'{global_name}{link}'
    response_last = requests.get(main_link_last, headers = header).text
    html_last = bs(response_last,'lxml')
    
    names = html_last.findAll('div',{'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
    names_list = []
    for i in names:
        names_list.append(i.getText())

    salarys_all = html_last.findAll('span',{'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'})
    for i in range(len(salarys_all)):
        salarys_all[i] = salarys_all[i].getText()   
    
    salarys_1 = []
    salarys_2 = []
    for i in range(len(salarys_all)):
        if salarys_all[i]:
            hm = re.findall(r'^от_*', salarys_all[i])
            hm2 = re.findall(r'^до_*', salarys_all[i])
            hm3 = re.search(r'—', salarys_all[i])
            hm4 = re.search(r'По договорённости', salarys_all[i])
            if hm:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'от', '', hm_num)
                salarys_1.append(hm_num)
                salarys_2.append('None')
            elif hm2:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                hm_num = re.sub(r'до', '', hm_num)
                salarys_1.append('None')
                salarys_2.append(hm_num)
            elif hm3:
                hm_num = re.split(r'—', salarys_all[i])
                hm_num1 = re.sub(r'\xa0', '', hm_num[0])
                hm_num2 = re.sub(r'\xa0', '', hm_num[1])
                salarys_1.append(hm_num1)
                salarys_2.append(hm_num2)
            elif hm4:
                salarys_1.append('None')
                salarys_2.append('None')
            else:
                hm_num = re.sub(r'\xa0', '', salarys_all[i])
                salarys_1.append(hm_num)
                salarys_2.append(hm_num)
                
    links_page = []
    links_page1 = html_last.findAll('a',{'class':'_1QIBo'})
    for i in links_page1:
        links_page2 = i['href']
        links_page.append(f'{global_name}{links_page2}')
    
    result['names'] = names_list
    result['salarys_1'] = salarys_1
    result['salarys_2'] = salarys_2
    result['links'] = links_page
    result['from'] = global_name
    
    return result

In [11]:
response_two = requests.get(main_link_two, headers = header).text
html_two = bs(response_two,'lxml')

In [12]:
pager_two = html_two.findAll('div',{'class':'L1p51'})
last_page_two = pager_two[0].findAll('span',{'class':'_3IDf-'})
link_last_num_two = last_page_two[-2].getText()

In [13]:
page_other_two = pd.DataFrame(columns=['names', 'salarys_1', 'salarys_2', 'links', 'from'])
for i in range(1, int(link_last_num_two)+1):
    link_last = f'/vakansii/{search_two}.html?geo%5Bt%5D%5B0%5D=4&page={i}'
    result = page_table_two(link_last)
    page_other_two = pd.merge(page_other_two, result, how='outer')

In [14]:
page_other

,names,salarys_1,salarys_2,links,from
0,Врач стоматолог-терапевт,70000,120000 руб.,https://hh.ru/vacancy/35721761?query=%D0%B2%D1...,https://hh.ru
1,Врач стоматолог-терапевт,70000,100000 руб.,https://hh.ru/vacancy/35705237?query=%D0%B2%D1...,https://hh.ru
2,Ассистент врача стоматолога- хирурга,None,None,https://hh.ru/vacancy/35113974?query=%D0%B2%D1...,https://hh.ru
3,Врач стоматолог (терапевт-хирург),50000,100000 руб.,https://hh.ru/vacancy/35328784?query=%D0%B2%D1...,https://hh.ru
4,Врач стоматолог универсал,100000 руб.,None,https://hh.ru/vacancy/35507904?query=%D0%B2%D1...,https://hh.ru
...,...,...,...,...,...
212,Врач-стоматолог-ортодонт,None,None,https://hh.ru/vacancy/35250500?query=%D0%B2%D1...,https://hh.ru
213,Врач-стоматолог,40000 руб.,None,https://hh.ru/vacancy/35250093?query=%D0%B2%D1...,https://hh.ru
214,Врач-стоматолог,None,None,https://hh.ru/vacancy/35249596?query=%D0%B2%D1...,https://hh.ru
215,Врач стоматолог-терапевт,100000,150000 руб.,https://hh.ru/vacancy/34450365?query=%D0%B2%D1...,https://hh.ru


In [15]:
page_other_two

,names,salarys_1,salarys_2,links,from
0,Медицинская сестра врача-стоматолога,None,None,https://superjob.ru/vakansii/medicinskaya-sest...,https://superjob.ru
1,Врач стоматолог-ортопед,None,None,https://superjob.ru/vakansii/vrach-stomatolog-...,https://superjob.ru
2,Ассистент врача-стоматолога,50000₽,None,https://superjob.ru/vakansii/assistent-vracha-...,https://superjob.ru
3,Врач стоматолог-терапевт-хирург (+детский стом...,None,None,https://superjob.ru/vakansii/vrach-stomatolog-...,https://superjob.ru
4,Врач стоматолог-терапевт,60000₽,None,https://superjob.ru/vakansii/vrach-stomatolog-...,https://superjob.ru
...,...,...,...,...,...
130,"Стоматолог-терапевт, эндодонтист (смешанный пр...",None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru
131,Рентгенолаборант в детскую стоматологию,45000,50000₽,https://superjob.ru/vakansii/rentgenolaborant-...,https://superjob.ru
132,Стоматолог-терапевт,None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru
133,"Стоматолог-терапевт (детский, взрослый)",None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru


In [16]:
page_other_all = pd.merge(page_other, page_other_two, how='outer')

In [17]:
page_other_all

,names,salarys_1,salarys_2,links,from
0,Врач стоматолог-терапевт,70000,120000 руб.,https://hh.ru/vacancy/35721761?query=%D0%B2%D1...,https://hh.ru
1,Врач стоматолог-терапевт,70000,100000 руб.,https://hh.ru/vacancy/35705237?query=%D0%B2%D1...,https://hh.ru
2,Ассистент врача стоматолога- хирурга,None,None,https://hh.ru/vacancy/35113974?query=%D0%B2%D1...,https://hh.ru
3,Врач стоматолог (терапевт-хирург),50000,100000 руб.,https://hh.ru/vacancy/35328784?query=%D0%B2%D1...,https://hh.ru
4,Врач стоматолог универсал,100000 руб.,None,https://hh.ru/vacancy/35507904?query=%D0%B2%D1...,https://hh.ru
...,...,...,...,...,...
347,"Стоматолог-терапевт, эндодонтист (смешанный пр...",None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru
348,Рентгенолаборант в детскую стоматологию,45000,50000₽,https://superjob.ru/vakansii/rentgenolaborant-...,https://superjob.ru
349,Стоматолог-терапевт,None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru
350,"Стоматолог-терапевт (детский, взрослый)",None,None,https://superjob.ru/vakansii/stomatolog-terape...,https://superjob.ru
